In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from wmfdata import charting, mariadb, hive

In [3]:
%matplotlib inline
mpl.style.use(charting.mpl_style)

The last data problem to be fixed was logging `VisualEditorFeatureUse` events on mobile. That finished rolling out of 7 February, so we can collect 3 full weeks of good data using the range 2019-02-08 ≤ dt < 2019-03-02.

In [4]:
# Start included, end excluded
DATA_START = "2019-02-08"
DATA_END = "2019-03-02"
PARTITION_CONDITION = "year = 2019 and month >= 2"

# Events

In [8]:
events_r = hive.run("""
select
    event.editing_session_id as editing_session_id,
    dt as timestamp,
    event.action as action,
    coalesce(
        event.init_type,
        event.save_failure_type,
        event.abort_type
    ) as type,
    coalesce(
        event.init_mechanism,
        event.abort_mechanism
    ) as mechanism,
    coalesce(
        event.ready_timing, 
        event.loaded_timing,
        event.abort_timing,
        event.save_intent_timing,
        event.save_attempt_timing,
        event.save_failure_timing,
        event.save_success_timing
    ) as timing
from event_sanitized.editattemptstep
where
    dt between "{data_start}" and "{data_end}" and
    {partition_condition} and 
    not event.is_oversample
""".format(
    data_start=DATA_START,
    data_end=DATA_END,
    partition_condition=PARTITION_CONDITION
))

events = events_r.copy()

In [9]:
events.info()
events.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13782557 entries, 0 to 13782556
Data columns (total 6 columns):
editing_session_id    object
timestamp             object
action                object
type                  object
mechanism             object
timing                float64
dtypes: float64(1), object(5)
memory usage: 630.9+ MB


In [16]:
# Make action a categorical variable with a roughly progressive sort order
action_names = ["init", "loaded", "ready", "saveIntent", "saveAttempt", "saveSuccess", "saveFailure", "abort"]

events = (
    events
    .assign(
        action=pd.Categorical(events["action"], categories=action_names, ordered=True),
        timestamp=pd.to_datetime(events["timestamp"]),
        type=pd.Categorical(events["type"]),
        mechanism=pd.Categorical(events["mechanism"])
    )
)

In [17]:
events.info()
events.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13782557 entries, 0 to 13782556
Data columns (total 6 columns):
editing_session_id    object
timestamp             datetime64[ns, UTC]
action                category
type                  category
mechanism             category
timing                float64
dtypes: category(3), datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 354.9+ MB


In [19]:
events.to_parquet("data/events.parquet")

# Feature uses

In [22]:
%%time
feature_uses = hive.run("""
select
    dt as timestamp,
    event.editingsessionid as editing_session_id,
    event.feature as feature,
    event.action as action
from event.visualeditorfeatureuse
where
    dt between "{data_start}" and "{data_end}" and
    {partition_condition}
""".format(
    data_start=DATA_START,
    data_end=DATA_END,
    partition_condition=PARTITION_CONDITION
)).assign(
    timestamp=lambda df: pd.to_datetime(df["timestamp"]),
    feature=lambda df: pd.Categorical(df["feature"]),
    action=lambda df: pd.Categorical(df["action"])
)

CPU times: user 10.7 s, sys: 48 ms, total: 10.7 s
Wall time: 2min 51s


In [30]:
feature_uses = (
    feature_uses.assign(
        timestamp=lambda df: df["timestamp"].dt.tz_convert(None)
    )
)

In [38]:
feature_uses.head()
feature_uses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350913 entries, 0 to 350912
Data columns (total 4 columns):
timestamp             350913 non-null datetime64[ns]
editing_session_id    350913 non-null object
feature               350913 non-null category
action                350913 non-null category
dtypes: category(2), datetime64[ns](1), object(1)
memory usage: 6.0+ MB


In [34]:
feature_uses.to_parquet("data/feature_uses.parquet")

# Sessions

In [37]:
sessions_r = hive.run("""
select
    event.editing_session_id as editing_session_id,
    wiki,
    event.platform as platform,
    event.integration as integration,
    collect_set(event.editor_interface) as editor,
    event.user_class as user_class,
    event.page_ns as page_ns,
    min(event.user_editcount) as edit_count,
    sum(cast(event.action = "init" as int)) as init_count,
    sum(cast(event.action = "loaded" as int)) as loaded_count,
    sum(cast(event.action = "ready" as int)) as ready_count,
    sum(cast(event.action = "saveIntent" as int)) as save_intent_count,
    sum(cast(event.action = "saveAttempt" as int)) as save_attempt_count,
    sum(cast(event.action = "saveSuccess" as int)) as save_success_count,
    sum(cast(event.action = "abort" as int)) as abort_count,
    max(dt) as max_timestamp,
    min(dt) as min_timestamp
from event.editattemptstep
where
    dt between "{data_start}" and "{data_end}" and
    {partition_condition} and 
    not event.is_oversample
group by 
    event.editing_session_id, 
    wiki,
    event.platform, 
    event.integration,
    event.user_class,
    event.page_ns
""".format(
    data_start=DATA_START,
    data_end=DATA_END,
    partition_condition=PARTITION_CONDITION
))

sessions = sessions_r.copy()

In [39]:
sessions.info()
sessions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5430481 entries, 0 to 5430480
Data columns (total 17 columns):
editing_session_id    object
wiki                  object
platform              object
integration           object
editor                object
user_class            object
page_ns               int64
edit_count            int64
init_count            int64
loaded_count          int64
ready_count           int64
save_intent_count     int64
save_attempt_count    int64
save_success_count    int64
abort_count           int64
max_timestamp         object
min_timestamp         object
dtypes: int64(9), object(8)
memory usage: 704.3+ MB


,editing_session_id,wiki,platform,integration,editor,user_class,page_ns,edit_count,init_count,loaded_count,ready_count,save_intent_count,save_attempt_count,save_success_count,abort_count,max_timestamp,min_timestamp
0,00001f404db876c6789065f799b8c845,kbpwiki,desktop,page,"[""wikitext""]",IP,0,0,1,0,0,0,0,0,0,2019-03-01T19:25:22Z,2019-03-01T19:25:22Z
1,000044d09162551a5632ce6e5984a8a5,enwiki,desktop,page,"[""wikitext""]",None,0,5103,0,3,3,0,1,1,0,2019-02-28T20:32:51Z,2019-02-28T20:30:34Z
2,00009cb0012114e6612677ac1b4000c2,enwiki,desktop,page,"[""wikitext""]",IP,0,0,1,0,0,0,0,0,0,2019-02-25T08:44:02Z,2019-02-25T08:44:02Z
3,0000a5305154a86b70b4d40a1216ed46,kaawiki,desktop,page,"[""wikitext""]",IP,3,0,1,0,0,0,0,0,0,2019-02-22T17:56:49Z,2019-02-22T17:56:49Z
4,0000bc70f61bcbb6acdd8e23b04dbf4e,enwiki,desktop,page,"[""wikitext""]",IP,0,0,1,0,0,0,0,0,0,2019-02-10T16:17:04Z,2019-02-10T16:17:04Z


Let's do some data wrangling!

We also have a lot of editors. The data is correct, but we're not interested in most.

In [40]:
pd.pivot_table(
    sessions,
    values="editing_session_id",
    index="editor",
    columns="platform",
    aggfunc=len,
    fill_value=0
)

platform,desktop,phone
editor,,
"[""visualeditor"",""wikitext""]",0,4146
"[""visualeditor"",""wikitext-2017""]",266,0
"[""visualeditor""]",239030,16935
"[""wikitext"",""visualeditor""]",0,7009
"[""wikitext""]",3670965,1481382
"[""wikitext-2017"",""visualeditor""]",304,0
"[""wikitext-2017""]",10444,0


The 2017 wikitext editor is a desktop-only opt-in beta feature, and is not widely used. We can remove the sessions that involved it.

In [41]:
nwe_rows = sessions.query("editor.str.contains('wikitext-2017')").index
sessions = sessions.drop(nwe_rows)

The phone sessions that mix the visual and wikitext editors are more important, because mobile VE is the focus on this project and we don't want to throw away any data about it. You can't switch between the two mobile editors once you make any changes, so we can treat the last editor in the session as the true editor.

While we're at it, let's also convert the array string syntax to something plainer.

In [42]:
sessions["editor"] = sessions["editor"].replace(to_replace={
    '["visualeditor","wikitext"]': "wikitext",
    '["wikitext","visualeditor"]': "visualeditor",
    '["visualeditor"]': "visualeditor",
    '["wikitext"]': "wikitext"
})

Things look a lot simpler now!

In [43]:
pd.pivot_table(
    sessions,
    values="editing_session_id",
    index="editor",
    columns="platform",
    aggfunc=len,
    fill_value=0
)

platform,desktop,phone
editor,,
visualeditor,239030,23944
wikitext,3670965,1485528


In [44]:
ec_bins = [0, 1, 10, 100, 1000, np.inf]
ec_labels = ["0 edits (new editor)", "1-9 edits", "10-99 edits", "100-999 edits", "1000+ edits"]

binned = pd.cut(sessions["edit_count"], bins=ec_bins, right=False, labels=ec_labels)
new_cats = binned.cat.categories.copy().tolist()
new_cats.insert(0, "IP")

# Override edit bin if the class is IP
ips = sessions.query("user_class == 'IP'")["user_class"]
    
# Combine into a categorical
combined = ips.combine_first(binned)
sessions = (
    sessions
    .assign(
        user_experience=pd.Categorical(combined, categories=new_cats, ordered=True)
    ).drop(
        ["user_class", "edit_count"],
        axis=1
    )
)

In [52]:
sessions = (
    sessions
    .assign(
        min_timestamp=pd.to_datetime(sessions["min_timestamp"]).dt.tz_localize(None),
        max_timestamp=pd.to_datetime(sessions["max_timestamp"]).dt.tz_localize(None),
        wiki=pd.Categorical(sessions["wiki"]),
        platform=pd.Categorical(sessions["platform"]),
        integration=pd.Categorical(sessions["integration"])
    )
    .assign(
        duration=lambda df: (df["max_timestamp"] - df["min_timestamp"]).dt.total_seconds().astype("int64")
    )
)

In [53]:
sessions.info()
sessions.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5419467 entries, 0 to 5430480
Data columns (total 17 columns):
editing_session_id    object
wiki                  category
platform              category
integration           category
editor                object
page_ns               int64
init_count            int64
loaded_count          int64
ready_count           int64
save_intent_count     int64
save_attempt_count    int64
save_success_count    int64
abort_count           int64
max_timestamp         datetime64[ns]
min_timestamp         datetime64[ns]
user_experience       category
duration              int64
dtypes: category(4), datetime64[ns](2), int64(9), object(2)
memory usage: 604.7+ MB


,editing_session_id,wiki,platform,integration,editor,page_ns,init_count,loaded_count,ready_count,save_intent_count,save_attempt_count,save_success_count,abort_count,max_timestamp,min_timestamp,user_experience,duration
0,00001f404db876c6789065f799b8c845,kbpwiki,desktop,page,wikitext,0,1,0,0,0,0,0,0,2019-03-01 19:25:22,2019-03-01 19:25:22,IP,0
1,000044d09162551a5632ce6e5984a8a5,enwiki,desktop,page,wikitext,0,0,3,3,0,1,1,0,2019-02-28 20:32:51,2019-02-28 20:30:34,1000+ edits,137
2,00009cb0012114e6612677ac1b4000c2,enwiki,desktop,page,wikitext,0,1,0,0,0,0,0,0,2019-02-25 08:44:02,2019-02-25 08:44:02,IP,0
3,0000a5305154a86b70b4d40a1216ed46,kaawiki,desktop,page,wikitext,3,1,0,0,0,0,0,0,2019-02-22 17:56:49,2019-02-22 17:56:49,IP,0
4,0000bc70f61bcbb6acdd8e23b04dbf4e,enwiki,desktop,page,wikitext,0,1,0,0,0,0,0,0,2019-02-10 16:17:04,2019-02-10 16:17:04,IP,0


In [54]:
sessions.to_parquet("data/sessions.parquet")

# Extracting and loading change tags

In [ ]:
# Not taking into account deleted edits!
TAG_START = "201710"
TAG_END = "201810"

tags = mariadb.multirun("""
select
    database() as wiki,
    rev_id,
    sum(ct_tag = "mobile web edit") as mobile_web,
    sum(ct_tag = "visualeditor") as visual_editor
from change_tag
inner join revision
on ct_rev_id = rev_id
where 
    ct_tag in ("mobile web edit", "visualeditor") and
    rev_timestamp between "{start}" and "{end}"
group by rev_id
""".format(start=TAG_START, end=TAG_END))

In [ ]:
tags["mobile_web"] = tags["mobile_web"].astype(bool)
tags["visual_editor"] = tags["visual_editor"].astype(bool)

In [ ]:
tags.head()

In [ ]:
def compute_editor(row):
    if row["mobile_web"]:
        if row["visual_editor"]:
            return "mobile visual editor"
        else:
            return "mobile wikitext editor"
    else:
        # Edits made with the desktop wikitext editors aren't included in this dateset
        return "desktop visual editor"
        
tags["editor"] = tags.apply(compute_editor, axis=1)
tags.head()

In [ ]:
tags.drop(
    labels=["mobile_web", "visual_editor"],
    axis=1
).to_csv(5
    "data/mob_or_ve_edits.tsv",
    sep="\t",
    index=False,
    header=False
)

In [ ]:
hive.run("""
drop table neilpquinn.mob_or_ve_edits
""")

In [ ]:
hive.run("""
create table neilpquinn.mob_or_ve_edits (
    wiki string,
    rev_id int,
    editor string
)
row format delimited fields terminated by '\t'
""")

In [ ]:
!hive -e "LOAD DATA LOCAL INPATH '/home/neilpquinn-wmf/proj/2018-08-Editing-metrics-snapshots/data/mob_or_ve_edits.tsv' OVERWRITE INTO TABLE neilpquinn.mob_or_ve_edits"

In [ ]:
hive.run("select * from neilpquinn.mob_or_ve_edits limit 10")

In [ ]:
hive.run("select count(*) from neilpquinn.mob_or_ve_edits")